<a href="https://colab.research.google.com/github/lovennut0/AI-waste-sorting/blob/main/AIforwastesorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Dataset_fixed.zip to Dataset_fixed.zip


for unzip .Zip file

In [ ]:
from google.colab import files
import zipfile

with zipfile.ZipFile("Dataset_fixed.zip", 'r') as zip_ref:
    zip_ref.extractall("Dataset_fixed")

test unzip file

In [ ]:
import os
print(os.listdir("Dataset_fixed"))

['glass', 'lightbulb', 'textile', 'battery', 'waterbottle', 'mask', 'paper', 'Cardboard', 'ointment', 'mobile', 'gloves', 'pills']


access to google drive (not needed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Train model code

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import json, os

dataset_dir = "Dataset_fixed"
img_size = 160
batch_size = 32
seed = 42

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Train generator WITH augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

# Exclude the __MACOSX directory
train_gen = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=seed,
    # Exclude __MACOSX directory
    classes=[d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d)) and d != '__MACOSX']
)

val_gen = val_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False,
    seed=seed,
    # Exclude __MACOSX directory
    classes=[d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d)) and d != '__MACOSX']
)

os.makedirs("artifacts", exist_ok=True)
with open("artifacts/class_indices.json", "w") as f:
    json.dump(train_gen.class_indices, f, indent=2)

base_model = tf.keras.applications.MobileNetV2(
    weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3)
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(img_size, img_size, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(train_gen.num_classes, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6, verbose=1)
ckpt = ModelCheckpoint("artifacts/waste_model_best1.h5", monitor="val_loss",
                       save_best_only=True, verbose=1)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=[early_stopping, reduce_lr, ckpt]
)

base_model.trainable = True
for layer in base_model.layers[:-40]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),  # lower LR
              loss="categorical_crossentropy",
              metrics=["accuracy"])

history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[early_stopping, reduce_lr, ckpt]
)

model.save("artifacts/waste_model_final1.h5")
print("✅ Training complete. Best model: artifacts/waste_model_best1.h5")

Found 1917 images belonging to 12 classes.
Found 474 images belonging to 12 classes.
Epoch 1/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3885 - loss: 2.0068
Epoch 1: val_loss improved from inf to 0.75790, saving model to artifacts/waste_model_best1.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - accuracy: 0.3917 - loss: 1.9962 - val_accuracy: 0.7785 - val_loss: 0.7579 - learning_rate: 0.0010
Epoch 2/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8445 - loss: 0.5242
Epoch 2: val_loss improved from 0.75790 to 0.61590, saving model to artifacts/waste_model_best1.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.8446 - loss: 0.5238 - val_accuracy: 0.7911 - val_loss: 0.6159 - learning_rate: 0.0010
Epoch 3/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8838 - loss: 0.3477
Epoch 3: val_loss improved from 0.61590 to 0.58050, saving model to artifacts/waste_model_best1.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.8840 - loss: 0.3475 - val_accuracy: 0.8165 - val_loss: 0.5805 - learning_rate: 0.0010
Epoch 4/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9247 - loss: 0.2511
Epoch 4: val_loss improved from 0.58050 to 0.53539, saving model to artifacts/waste_model_best1.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.9246 - loss: 0.2516 - val_accuracy: 0.8312 - val_loss: 0.5354 - learning_rate: 0.0010
Epoch 5/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9248 - loss: 0.2402
Epoch 5: val_loss improved from 0.53539 to 0.52787, saving model to artifacts/waste_model_best1.h5


...

60/60 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.9299 - loss: 0.2140 - val_accuracy: 0.8312 - val_loss: 0.5026 - learning_rate: 0.0010
Epoch 7/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9474 - loss: 0.1741
Epoch 7: val_loss did not improve from 0.50259
60/60 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.9473 - loss: 0.1743 - val_accuracy: 0.8228 - val_loss: 0.5214 - learning_rate: 0.0010
Epoch 8/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9447 - loss: 0.1741
Epoch 8: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 8: val_loss did not improve from 0.50259
60/60 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.9446 - loss: 0.1743 - val_accuracy: 0.8397 - val_loss: 0.5130 - learning_rate: 0.0010
Epoch 9/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9449 - loss: 0.1734
Epoch 9: val_loss did not improve from 0.50259
60/60 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.9450 - loss: 0.1732 - val_accuracy: 0.8207 - val_loss: 0.5287 -

60/60 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.9456 - loss: 0.1508 - val_accuracy: 0.8376 - val_loss: 0.4984 - learning_rate: 2.5000e-04
Epoch 12/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9533 - loss: 0.1527
Epoch 12: val_loss did not improve from 0.49840
60/60 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.9532 - loss: 0.1529 - val_accuracy: 0.8228 - val_loss: 0.5033 - learning_rate: 2.5000e-04
Epoch 13/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9574 - loss: 0.1271
Epoch 13: val_loss improved from 0.49840 to 0.48829, saving model to artifacts/waste_model_best1.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.9574 - loss: 0.1273 - val_accuracy: 0.8249 - val_loss: 0.4883 - learning_rate: 2.5000e-04
Epoch 14/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9539 - loss: 0.1423
Epoch 14: val_loss did not improve from 0.48829
60/60 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.9540 - loss: 0.1421 - val_accuracy: 0.8270 - val_loss: 0.4893 - learning_rate: 2.5000e-04
Epoch 15/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9496 - loss: 0.1399
Epoch 15: val_loss improved from 0.48829 to 0.46698, saving model to artifacts/waste_model_best1.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.9496 - loss: 0.1398 - val_accuracy: 0.8439 - val_loss: 0.4670 - learning_rate: 2.5000e-04
Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8604 - loss: 0.3848
Epoch 1: val_loss did not improve from 0.46698
60/60 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step - accuracy: 0.8610 - loss: 0.3836 - val_accuracy: 0.8207 - val_loss: 0.5836 - learning_rate: 1.0000e-04
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9438 - loss: 0.1775
Epoch 2: val_loss did not improve from 0.46698
60/60 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - accuracy: 0.9439 - loss: 0.1772 - val_accuracy: 0.8186 - val_loss: 0.6144 - learning_rate: 1.0000e-04
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9743 - loss: 0.0898
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 3: val_loss did not improve from 0.46698
60/60 ━━━━━━━━━━━━━━━━━━━━ 139s 2s/step - accuracy: 0.9742 - loss: 0.0900 - val_accuracy: 0.8207 - val_

✅ Training complete. Best model: artifacts/waste_model_best1.h5


Evaluate Validation

In [ ]:
import tensorflow as tf


model = tf.keras.models.load_model("artifacts/waste_model_best1.h5")

loss, accuracy = model.evaluate(val_gen)

print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

Test model code (1 pic)

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from google.colab import files

# Load the trained model
model = tf.keras.models.load_model("artifacts/waste_model_final1.h5")

# Load class labels
# Make sure 'train_gen' or equivalent is available or load class_indices.json
# Assuming 'train_gen' is available from previous cell execution
try:
    classes = list(train_gen.class_indices.keys())
except NameError:
    import json
    with open("artifacts/class_indices.json", "r") as f:
        class_indices = json.load(f)
    classes = list(class_indices.keys())

print("Class labels:", classes)

uploaded = files.upload()
img_path = list(uploaded.keys())[0]

# Preprocess image
img = cv2.imread(img_path)
# Resize the image to the size the model expects (160x160)
img_resized = cv2.resize(img, (160, 160))
img_normalized = img_resized.astype(np.float32) / 255.0
img_input = np.expand_dims(img_normalized, axis=0)

preds = model.predict(img_input)
class_id = np.argmax(preds)
print("✅ Prediction:", classes[class_id], "| Confidence:", preds[0][class_id])

Class labels: ['glass', 'lightbulb', 'textile', 'battery', 'waterbottle', 'mask', 'paper', 'Cardboard', 'ointment', 'mobile', 'gloves', 'pills']


Saving BS-1500-single-Mockup_5000x.webp to BS-1500-single-Mockup_5000x (1).webp
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
✅ Prediction: waterbottle | Confidence: 0.90756434


test model (multiple pictures)

In [ ]:

model = tf.keras.models.load_model("waste_model.h5")

classes = ["battery", "Cardboard", "glass", "gloves","lightbulb","mask","mobile","ointment","paper","pills","PlasticBottle","spraycan","textile","thermometer","tin_steelcans"]  # replace class with folder names
print("Class labels:", classes)

uploaded = files.upload()

for filename in uploaded.keys():
    img = cv2.imread(filename)
    img_resized = cv2.resize(img, (128,128))
    img_normalized = img_resized.astype(np.float32) / 255.0
    img_input = np.expand_dims(img_normalized, axis=0)

    preds = model.predict(img_input)
    class_id = np.argmax(preds)
    print(f"✅ {filename} → {classes[class_id]} | Confidence: {preds[0][class_id]:.4f}")

In [ ]:
!rm -rf /content/Dataset
!rm -rf /content/artifacts/
!rm -rf /content/sample_data